<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/Leonardo_Pacheco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = 'Leonardo Augusto da Silva Pacheco'
print(f'Meu nome é {nome}.')

Meu nome é Leonardo Augusto da Silva Pacheco.


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.9 MB/s 
     |████████████████████████████████| 596 kB 71.1 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 6.6 MB 71.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 14:20:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.examples = []
        for text in tqdm_notebook(texts):
            token_ids = tokenize(text=text, tokenizer=tokenizer)
            if len(token_ids) < context_size + 1:
                continue
            # Compute n-grams:
            for i in range(len(token_ids) - context_size):
                input_ids = token_ids[i:i + context_size]
                target_id = token_ids[i + context_size]
                self.examples.append((input_ids, target_id)) 

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids, target_id = self.examples[idx]
        return torch.LongTensor(input_ids), target_id

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


## Inicialização do Neptune

In [ ]:
!pip install -U neptune-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 7.5 MB/s 
     |████████████████████████████████| 829 kB 71.3 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 181 kB 70.9 MB/s 
     |████████████████████████████████| 132 kB 96.0 MB/s 
     |████████████████████████████████| 79 kB 10.7 MB/s 
     |████████████████████████████████| 8.8 MB 60.3 MB/s 
     |████████████████████████████████| 138 kB 105.8 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 127 kB 83.6 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 130 kB 83.5 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.16.2-py2.py3-none-any.whl size=566337 sha256=ff678b046738bc503cdbc96ac14beabb684765bcfc0e9dd59

In [ ]:
import neptune.new as neptune

run = neptune.init(
    project="leonardo3108/IA025Aula8",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhZDJhYWRmZi0zZmE0LTRhYzAtYThlMS1iYmJjMzU1NWU5YzQifQ==",
)

https://app.neptune.ai/leonardo3108/IA025Aula8/e/IA025AULA8-8
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-25 14:20:58--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.195.128, 142.250.99.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M  50.9MB/s    in 2.3s    

2022-05-25 14:21:00 (50.9 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [19]:
# Load datasets
context_size = 9

valid_examples = 1000
test_examples = 2000
texts = open('sample_brwac.txt').readlines()

#print('Truncating for debugging purposes.')
#texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

texts = training_texts = valid_texts = test_texts = None

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/22000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [20]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 24392506
valid examples: 1290007
test examples: 2242228


In [21]:
from torch.nn import Embedding, Sequential, Linear, Flatten, ReLU, Softmax, LayerNorm
from torch import arange, div, matmul, transpose, bmm

from collections import OrderedDict

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()
        self.context_size = context_size
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        hidden_size = 2048

        self.W_c = Embedding(vocab_size, embedding_dim)
        self.W_p = Embedding(context_size, embedding_dim) 
        self.W_q = Linear(embedding_dim, embedding_dim, bias=False)
        self.W_k = Linear(embedding_dim, embedding_dim, bias=False)
        self.W_v = Linear(embedding_dim, embedding_dim, bias=False)
        self.W_o = Linear(embedding_dim, embedding_dim, bias=False)
        

        self.dense = Sequential(
            Flatten(),
            LayerNorm(context_size * embedding_dim),
            Linear(context_size * embedding_dim, hidden_size),
            ReLU(),
            Linear(hidden_size, vocab_size))

        self.softmax = Softmax(dim=-1)

    def forward(self, inputs, debug=False):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        if debug: print(f"batch size    (B): {inputs.shape[0]}")
        if debug: print(f"context size  (L): {self.context_size}")
        if debug: print(f"embedding dim (D): {self.embedding_dim}")
        if debug: print(f"vocab size    (V): {self.vocab_size}")
        if debug: print()

        positions = arange(self.context_size, device=inputs.device).unsqueeze(0)
        if debug: print(f"inputs: {inputs.shape}")
        if debug: print(f"positions: {positions.shape}")

        C = self.W_c(inputs)
        P = self.W_p(positions)
        X = C + P
        if debug: print(f"C: {C.shape}")
        if debug: print(f"P: {P.shape}")
        if debug: print(f"X: {X.shape}")

        Q = self.W_q(X)
        K = self.W_k(X)
        V = self.W_v(X)
        O = self.W_o(X)
        if debug: print(f"Q: {Q.shape}")
        if debug: print(f"K: {K.shape}")
        if debug: print(f"V: {V.shape}")
        if debug: print(f"O: {O.shape}")
        if debug: print()

        scores = bmm(Q, transpose(K, 1, -1))
        probs = self.softmax(scores)
        E = bmm(probs, V)
        E *= O

        if debug: print(f"scores: {scores.shape}")
        if debug: print(f"probs: {probs.shape}")
        if debug: print(f"E: {E.shape}")

        E += X   # residual

        logits = self.dense(E)
        return logits

## Teste o modelo com um exemplo

In [22]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
#model(sample_train_gpu).shape
logits = model.forward(sample_train_gpu, debug=True)
print('Logits:', logits.shape)

batch size    (B): 1
context size  (L): 9
embedding dim (D): 64
vocab size    (V): 29794

inputs: torch.Size([1, 9])
positions: torch.Size([1, 9])
C: torch.Size([1, 9, 64])
P: torch.Size([1, 9, 64])
X: torch.Size([1, 9, 64])
Q: torch.Size([1, 9, 64])
K: torch.Size([1, 9, 64])
V: torch.Size([1, 9, 64])
O: torch.Size([1, 9, 64])

scores: torch.Size([1, 9, 9])
probs: torch.Size([1, 9, 9])
E: torch.Size([1, 9, 64])
Logits: torch.Size([1, 29794])


In [23]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 64154530


## Assert da Perplexidade


In [24]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              31106
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [25]:
from copy import deepcopy

max_examples = 50_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

run['sys/tags'].add([f"model:AttentionLanguageModel"])
#run['parameters'] = params

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


n_examples = 0
step = 0
early_points = ''
min_ppl = float('inf')
while n_examples < max_examples and early_points != '*****':
    train_losses = []
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        run['train/batch_loss'].log(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))
            run['train/perplexity'].log(train_ppl)

            with torch.no_grad():
                valid_losses = []
                for input, target in validation_loader:
                    valid_loss = validation_step(input.to(device), target.to(device))
                    valid_losses.append(valid_loss)
                    run['valid/batch_loss'].log(valid_loss)
                valid_ppl = np.exp(np.average(valid_losses))
                run['valid/perplexity'].log(valid_ppl)

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}', early_points)
            train_losses = []

            if min_ppl <= valid_ppl:
                early_points += '*'
                if early_points == '*****':
                    print('Early stop!')
                    break
            else:
                early_points = ''
                min_ppl = valid_ppl
                best_model = deepcopy(model.state_dict())
                torch.save(best_model, 'best_model.pth')


        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

model.load_state_dict(best_model)

0 steps; 0 examples so far; train ppl: 29821.31, valid ppl: 27689.63
10000 steps; 639959 examples so far; train ppl: 1442.31, valid ppl: 1184.55
20000 steps; 1279918 examples so far; train ppl: 998.88, valid ppl: 914.65
30000 steps; 1919877 examples so far; train ppl: 829.53, valid ppl: 788.87
40000 steps; 2559836 examples so far; train ppl: 722.20, valid ppl: 710.79
50000 steps; 3199795 examples so far; train ppl: 660.82, valid ppl: 653.29
60000 steps; 3839754 examples so far; train ppl: 603.89, valid ppl: 599.11
70000 steps; 4479713 examples so far; train ppl: 562.19, valid ppl: 559.40
80000 steps; 5119672 examples so far; train ppl: 530.59, valid ppl: 534.05
90000 steps; 5759631 examples so far; train ppl: 519.30, valid ppl: 512.73
100000 steps; 6399590 examples so far; train ppl: 485.67, valid ppl: 492.43
110000 steps; 7039549 examples so far; train ppl: 467.46, valid ppl: 472.69
120000 steps; 7679508 examples so far; train ppl: 448.76, valid ppl: 455.86
130000 steps; 8319467 examp

<All keys matched successfully>

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [26]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 270.557827639406


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [27]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz a
Eu gosto de comer pizza pois me faz a ser
Eu gosto de comer pizza pois me faz a ser a
Eu gosto de comer pizza pois me faz a ser a primeira
Eu gosto de comer pizza pois me faz a ser a primeira vez
Eu gosto de comer pizza pois me faz a ser a primeira vez que
Eu gosto de comer pizza pois me faz a ser a primeira vez que o
Eu gosto de comer pizza pois me faz a ser a primeira vez que o governo
Eu gosto de comer pizza pois me faz a ser a primeira vez que o governo,
Eu gosto de comer pizza pois me faz a ser a primeira vez que o governo, o
Eu gosto de comer pizza pois me faz a ser a primeira vez que o governo, o que
Eu gosto de comer pizza pois me faz a ser a primeira vez que o governo, o que é
Eu gosto de comer pizza pois me faz a ser a primeira vez que o governo, o que é o
Eu gosto de comer pizza pois me faz a ser a primeira vez que o governo, o que é o que
Eu gosto de comer pizza pois me faz a ser a primeira vez que o governo, o que é o que é
Eu gosto de

In [ ]:


aa 